<a href="https://colab.research.google.com/github/kotha-karthik/AI_Projects/blob/main/AiX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit streamlit_option_menu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 19.7 MB/s eta 0:00:00


In [4]:
!pip install pyngrok streamlit
from pyngrok import ngrok
import streamlit as st

with open('app.py', 'w') as f:
    f.write('''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import streamlit as st
from streamlit_option_menu import option_menu
from transformers import pipeline

# Set page config for a better layout
st.set_page_config(page_title="AiXData Analytics Dashboard", layout="wide")

# Apply custom theme and styling
st.markdown("""
    <style>
    .main-header {
        text-align: center;
        font-family: 'Helvetica', sans-serif;
        font-size: 36px;
        color: #4B0082;
        padding: 20px;
    }
    .main-content {
        background-color: #F0F2F6;
        padding: 30px;
        border-radius: 10px;
        box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
    }
    </style>
""", unsafe_allow_html=True)

# Custom header
st.markdown("<div class='main-header'>Data Analytics Dashboard</div>", unsafe_allow_html=True)
st.markdown("""
    <div class='main-content'>
    <p>Upload your CSV file, run analytics, and enjoy beautiful, interactive visualizations.</p>
    <p>Get insights at your fingertips, powered by cutting-edge AI algorithms!</p>
    </div>
""", unsafe_allow_html=True)

# Sidebar for Navigation
with st.sidebar:
    selected = option_menu(
        "Main Menu", ["Home", "Data Analysis", "Visualization", "NLP Insights"],
        icons=["house", "graph-up", "bar-chart", "chat-left-text"],
        menu_icon="menu-app", default_index=0
    )

# Data Ingestion
def ingest_data(uploaded_file):
    if uploaded_file is not None:
        data = pd.read_csv(uploaded_file)
        return data
    else:
        st.error("Please upload a CSV file.")
        return None

# Data Cleaning
def clean_data(data):
    data.ffill(inplace=True)  # Forward fill for missing values
    label_encoder = LabelEncoder()
    for column in data.select_dtypes(include=['object']):
        data[column] = label_encoder.fit_transform(data[column])
    return data

# Descriptive Analytics
def descriptive_analysis(data):
    return data.describe(), data.corr()

# Predictive Analytics with multiple model options
def predictive_model(data, target_column, model_choice):
    X = data.drop(columns=[target_column])
    y = data[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Choose model
    if model_choice == "Random Forest Regressor":
        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return mean_squared_error(y_test, y_pred)

    elif model_choice == "Random Forest Classifier":
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted'), recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')

# Visualization with Plotly
def visualize_data(data, column, chart_type):
    if chart_type == "Bar":
        fig = px.bar(data, x=column, y=data.index)
    elif chart_type == "Line":
        fig = px.line(data, x=data.index, y=column)
    elif chart_type == "Heatmap":
        fig = px.imshow(data.corr(), text_auto=True)
    elif chart_type == "Pie":
        pie_data = data[column].value_counts()
        fig = px.pie(pie_data, values=pie_data, names=pie_data.index)

    st.plotly_chart(fig)

# Generate insights based on analysis
def generate_insights(data):
    insights = []
    if 'conversion_rate' in data.columns and 'traffic_source' in data.columns:
        avg_conversion = data['conversion_rate'].mean()
        insights.append(f"The average conversion rate is {avg_conversion:.2f}.")
    if 'bounce_rate' in data.columns:
        max_bounce = data['bounce_rate'].max()
        insights.append(f"The highest bounce rate is {max_bounce:.2f}.")
    return insights

# NLP model initialization
nlp = pipeline("question-answering")

# Answering user questions using NLP
def answer_question(question, context):
    result = nlp(question=question, context=context)
    return result['answer']

# Home Page
if selected == "Home":
    st.markdown("<h2 style='text-align: center;'>Welcome to AiXData Dashboard!</h2>", unsafe_allow_html=True)
    st.markdown("""
        <div style='margin: 20px; padding: 20px; background-color: #F7F7F7; border-radius: 10px; text-align: center;'>
        <p>Our platform allows seamless data analysis and visualization. Easily upload your data, and get actionable insights.</p>
        <p>Designed for financial companies to make data-driven decisions quickly.</p>
        </div>
    """, unsafe_allow_html=True)

# Data Analysis Page
if selected == "Data Analysis":
    st.header("Data Analysis")

    uploaded_file = st.file_uploader("Choose a CSV file", type="csv")
    if uploaded_file is not None:
        data = ingest_data(uploaded_file)
        cleaned_data = clean_data(data)

        st.write("Data Sample:", cleaned_data.head())
        desc_stats, correlation_matrix = descriptive_analysis(cleaned_data)

        st.write("Summary Statistics:")
        st.dataframe(desc_stats)
        st.write("Correlation Matrix:")
        st.dataframe(correlation_matrix)

        target_column = st.selectbox("Select Target Column", cleaned_data.columns)
        model_choice = st.selectbox("Select Model", ["Random Forest Regressor", "Random Forest Classifier"])

        if st.button("Run Predictive Model"):
            if model_choice == "Random Forest Regressor":
                result = predictive_model(cleaned_data, target_column, model_choice)
                st.write("Mean Squared Error:", result)
            else:
                acc, prec, recall, f1 = predictive_model(cleaned_data, target_column, model_choice)
                st.write("Accuracy:", acc)
                st.write("Precision:", prec)
                st.write("Recall:", recall)
                st.write("F1 Score:", f1)

        # Generate and display insights
        insights = generate_insights(cleaned_data)
        st.write("Key Insights:")
        for insight in insights:
            st.write(insight)

# Visualization Page
if selected == "Visualization":
    st.header("Data Visualization")

    uploaded_file = st.file_uploader("Choose a CSV file for visualization", type="csv")
    if uploaded_file is not None:
        data = ingest_data(uploaded_file)
        cleaned_data = clean_data(data)

        column_for_viz = st.selectbox("Select Column for Visualization", cleaned_data.columns)
        chart_type = st.selectbox("Select Chart Type", ["Bar", "Line", "Heatmap", "Pie"])

        if st.button("Visualize Data"):
            visualize_data(cleaned_data, column_for_viz, chart_type)

# NLP Insights Page
if selected == "NLP Insights":
    st.header("NLP for Predictive Questions")

    context = st.text_area("Provide context (e.g., insights from your dataset)", height=200)
    question = st.text_input("Ask a predictive question about your data")

    if st.button("Get Answer"):
        if context and question:
            answer = answer_question(question, context)
            st.write("Answer:", answer)
        else:
            st.error("Please provide both context and a question.")

# Footer Section
st.markdown("""
    <div style='margin: 20px; padding: 20px; background-color: #F7F7F7; border-radius: 10px; text-align: center;'>
    <p>Contact us: info@AiXData.com</p>
    <p>&copy; 2024 AiXData. All rights reserved.</p>
    </div>
""", unsafe_allow_html=True)

# Run Streamlit app
if __name__ == "__main__":
    st.markdown("### Thank you for using AiXData Analytics Dashboard!")







''')

# Run Streamlit app
if __name__ == "__main__":
    st.markdown("### Thank you for using the AiXData Analytics Dashboard!")
# Run Streamlit using ngrok

!ngrok authtoken 2jhYgyz8tELtO3SmgxkdtIuY6Hs_4qJe1oVLyynEZtqWZgaVG # Replace with your actual auth token
public_url = ngrok.connect(8501)
print(public_url)
!streamlit run app.py &


2024-10-29 13:20:33.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 13:20:33.576 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NgrokTunnel: "https://dbb7-34-32-128-149.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.32.128.149:8501

2024-10-29 13:20:57.684891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 13:20:57.713943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 13:20:57.722942: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 13:20:57.743927: I tensor